<a href="https://colab.research.google.com/github/braveenth/Canadian-Weather-AI/blob/main/Canadian_Weather_AI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Canadian Weather AI
By: Braveenth Rasanayagam

Multi-modal
- Vision
- Text to Voice

Cloud Run Jobs

In [2]:
image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"
print(image_url)
!curl -o case_image.png {image_url}

from IPython.display import Image
Image("/content/jet_stream.png")
Image(url=image_url)

https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  292k  100  292k    0     0   117k      0  0:00:02  0:00:02 --:--:--  117k


In [9]:
import os

try:
    # Attempt to import the google.colab module to see if the program is running in Colab.
    from google.colab import userdata
    print("Running on Google Colab")
    !pip install openai
    !pip install pydub
    !pip install anthropic
    from openai import OpenAI
    client = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))
    ANTHRONPIC_API_KEY = userdata.get('ANTHROPIC_API_KEY')
    !mkdir -p /content/text
    !mkdir -p /content/voice
    !mkdir -p /content/assets
    !curl -o "/content/assets/WeatherNetwork.mp3" "https://storage.googleapis.com/can-weather-ai/assets/WeatherNetwork.mp3"

except ImportError:
    # The ImportError exception will be raised if the google.colab module is not found,
    # indicating that the program is not running inside Google Colab.
    print("Running outside of Google Colab")
    from openai import OpenAI
    client = OpenAI(api_key=(os.environ.get('OPENAI_API_KEY')))
    ANTHRONPIC_API_KEY = os.environ.get('ANTHROPIC_API_KEY')


Running on Google Colab
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 64.1M  100 64.1M    0     0  15.4M      0  0:00:04  0:00:04 --:--:-- 15.4M


In [10]:
# This will be incorporated into the prompt in the future

canadianMetric = {}

canadianMetric[0] = "not Canadian"
canadianMetric[1] = "somewhat Canadian"
canadianMetric[2] = "slightly Canadian"
canadianMetric[3] = "moderately Canadian"
canadianMetric[4] = "very Canadian"
canadianMetric[5] = "as Canadian as possible"

selectedCanadianMetric = canadianMetric[4]
langauge = "en"

# We are allowing for multiple models to be incorporated
modelChoice = {}

modelChoice[0] = "gpt-4-vision-preview"
modelChoice[1] = "claude-3-opus-20240229"

selectedModel = modelChoice[1]

In [25]:
imagePrompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. Make it sound very Canadian. It should sound as Canadian as absolutely possible."
image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"

if selectedModel == "gpt-4-vision-preview":
  gpt_mode = "gpt-4-vision-preview"
  imagePrompt = "You are a Canadian weatherperson. Present this jet stream information to an audience. Make it sound very Canadian. It should sound as Canadian as absolutely possible."
  image_url = "https://weather.gc.ca/data/jet_stream/tempmapwx_e.gif"

  response = client.chat.completions.create(
    model="gpt-4-vision-preview",
    messages=[
      {
        "role": "user",
        "content": [
          {"type": "text", "text": imagePrompt},
          {
            "type": "image_url",
            "image_url": {
              "url": image_url,
            },
          },
        ],
      }
    ],
    max_tokens=600,
    temperature=0.7,
    top_p=1,
    frequency_penalty=0,
    presence_penalty=0
  )
  weatherScript = response.choices[0].message.content
  print(response.choices[0])

elif selectedModel == "claude-3-opus-20240229":
  import anthropic
  import base64
  import httpx
  claude_mode = "claude-3-opus-20240229"

  client = anthropic.Anthropic(
      api_key=ANTHRONPIC_API_KEY,
  )

  image_media_type = "image/gif"
  image_data = base64.b64encode(httpx.get(image_url).content).decode("utf-8")
  imagePrompt += "Do not put any statements with asterisks surrounding them. Only give the speech, and nothing else. For example, statements similar to the following should be excluded: *clears throat*"

  message = client.messages.create(
      model="claude-3-opus-20240229",
      max_tokens=1024,
      messages=[
          {
              "role": "user",
              "content": [
                  {
                      "type": "image",
                      "source": {
                          "type": "base64",
                          "media_type": image_media_type,
                          "data": image_data,
                      },
                  },
                  {
                      "type": "text",
                      "text": imagePrompt
                  }
              ],
          }
      ],
  )
  weatherScript = message.content[0].text
  print(weatherScript)


Ooo boy, would ya take a gander at this jet stream map, eh? We've got quite the situaaaation brewin' over the Great White North today!

Looks like a big ol' trough is dippin' down over BC and Alberta, bringin' some chilly arctic air. Meanwhile, a real beaut of a ridge is buildin' over Quebec and the Maritimes. That's gonna serve up some milder temps for the folks out east, dontcha know.

But hold onto yer toques! Smack dab in the middle, from Manitoba to Ontario, the jet stream's takin' a wild ride. It's zippin' and zappin' all over the place like a hockey player danglin' through traffic. That's a recipe for some unsettled weather with a chance of Timbits precipitation, I tell ya!

So there ya have it, my fellow Canucks. Keep your stick on the ice and your eyes on the skies. And remember, whether you're weatherin' a Prairie blizzard or soakin' up a wee bit of East Coast sun, we're all in this together. Stay warm out there and don't forget to top up yer double-double!

This has been yer

In [26]:
print(weatherScript)

Ooo boy, would ya take a gander at this jet stream map, eh? We've got quite the situaaaation brewin' over the Great White North today!

Looks like a big ol' trough is dippin' down over BC and Alberta, bringin' some chilly arctic air. Meanwhile, a real beaut of a ridge is buildin' over Quebec and the Maritimes. That's gonna serve up some milder temps for the folks out east, dontcha know.

But hold onto yer toques! Smack dab in the middle, from Manitoba to Ontario, the jet stream's takin' a wild ride. It's zippin' and zappin' all over the place like a hockey player danglin' through traffic. That's a recipe for some unsettled weather with a chance of Timbits precipitation, I tell ya!

So there ya have it, my fellow Canucks. Keep your stick on the ice and your eyes on the skies. And remember, whether you're weatherin' a Prairie blizzard or soakin' up a wee bit of East Coast sun, we're all in this together. Stay warm out there and don't forget to top up yer double-double!

This has been yer

In [27]:
# Save the weatherScript as a file
from datetime import datetime

# Adding timezone
import pytz
timezone = pytz.timezone('America/Toronto')
now_in_timezone = datetime.now(timezone)

# Format the date
date = now_in_timezone.strftime("%Y-%m-%d")

#date = datetime.now().strftime("%Y-%m-%d")
count = 0

while os.path.exists(f"/content/text/ai-canadian-jetstream-{date}_{count}.txt"):
    count += 1
file = open(f"/content/text/ai-canadian-jetstream-{date}_{count}.txt", "w")
file.write(weatherScript)
file.close()

In [28]:
from openai import OpenAI

try:
    # Attempt to import the google.colab module to see if the program is running in Colab.
    from google.colab import userdata
    print("Running on Google Colab")
    audioClient = OpenAI(api_key=userdata.get('OPENAI_API_KEY'))

except ImportError:
    # The ImportError exception will be raised if the google.colab module is not found,
    # indicating that the program is not running inside Google Colab.
    import os
    print("Running outside of Google Colab")
    audioClient = OpenAI(api_key=(os.environ.get('OPENAI_API_KEY')))

audioResponse = audioClient.audio.speech.create(
    model="tts-1",
    voice="onyx",
    input=weatherScript,
)

filename = f"weatherperson-output-{date}.mp3"

Running on Google Colab


In [29]:
audioResponse.stream_to_file(filename)

<ipython-input-29-02f298be9c94>:1: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  audioResponse.stream_to_file(filename)


In [30]:
from pydub import AudioSegment

# Export the result
weatherPersonFileCount = 0
while os.path.exists(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav"):
    weatherPersonFileCount += 1

weatherpersonFile = f"/content/{filename}-{count}"
audioResponse.stream_to_file(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav")

# Obtain Audio Files
audio_bg_music = AudioSegment.from_file("/content/assets/WeatherNetwork.mp3", format="mp3")
audio_weatherperson = AudioSegment.from_file(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav", format="mp3")

# BG Music Segments from https://www.youtube.com/watch?v=6E2uzYAGPQU
#####################################################################
# 01. Morning Report (1998) Conclusions 0:00. #
# 02. Autumn (1999) Flying V.1 1:59 #
# 03. Holidays (2000) Xmas Spirit 3:29
# 04. Biding My Time 4:59
# 05. Winter (2002-'03) Life to the Full 7:12
# 06. Spring-Autumn (2003) Healthy Outlook 8:42
# 07. Winter (2003-04) Embrace Life 10:43
# 08. Primary Theme (2004-05) Lazing on the Slopes 13:16
# 09. Theme (2005-06) Windstar 15:17
# 10. Theme (2006-10) 17:17
# 11. Theme (2010-201?) Song Contest Winner 19:37
# 12. Holiday Version of Theme 11 (2011) 21:58
# 13. 25th Anniversary (2004-05 Secondary Theme) 25:02

segments = [0] * 12
segments[0] = [0, 119000, 7]
segments[1] = [119000, 209000, 7]
segments[2] = [209000, 299000, 7]
segments[3] = [299000, 432000, 7]
segments[4] = [432000, 522000, 7]
segments[5] = [522000, 643000, 7]
segments[6] = [643000, 796000, 7]
segments[7] = [796000, 917000, 7]
segments[8] = [917000, 1037000, 7]
segments[9] = [1037000, 1177000, 7]
segments[10] = [1177000, 1318000, 11]
segments[11] = [1318000, 1502000, 7]

# Custom timing adjustments based on listening observations
segments[8][0] = segments[8][0] + 8000

# Define start and end times in milliseconds
# Start at 17:22
# End at 17:3
start_time = 1042000  # Start at 10 seconds
end_time = start_time + 120000

# Use a Random number between 0 an 11 to obtain the segment number
import random
segment_number = random.randint(0, 11)

# Using Segments Here
start_time = segments[segment_number][0] + 1000
end_time = segments[segment_number][1]

# Slice the audio segment to the desired part
specific_part = audio_bg_music[start_time:end_time]

# Lower the volume by 10 dB
specific_part_quieter = specific_part - segments[segment_number][2]

# TO-DO: if the audio_weatherperson audio length is longer than the specific_part_quieter, then add some padding to the end of the specific_part_quieter
lenBackgroundMusicMS = end_time - start_time
lenAudioWeatherPersonMS = len(audio_weatherperson)
if lenAudioWeatherPersonMS > lenBackgroundMusicMS:
    from pydub.generators import Silence
    padding_length_ms = lenAudioWeatherPersonMS - lenBackgroundMusicMS + 1000
    #padding = Silence().to_audio_segment(duration=padding_length_ms, frame_rate=44100)
    padding = AudioSegment.silent(duration=padding_length_ms)
    specific_part_quieter = specific_part_quieter.append(padding)

# TO_DO: Add audio ducking

# Overlay the audio files
# Here, overlay_audio will start at 0 milliseconds into base_audio
combined = specific_part_quieter.overlay(audio_weatherperson, position=0)

# Export the result
count = 0
while os.path.exists(f"/content/voice/ai-canadian-jetstream-{date}_{count}.mp3"):
    count += 1
combined.export(f"/content/voice/ai-canadian-jetstream-{date}_{count}.mp3", format="mp3")


<ipython-input-30-4ba39fd69ab6>:9: DeprecationWarning: Due to a bug, this method doesn't actually stream the response content, `.with_streaming_response.method()` should be used instead
  audioResponse.stream_to_file(f"/content/weatherperson-output-{date}_{weatherPersonFileCount}.wav")


<_io.BufferedRandom name='/content/voice/ai-canadian-jetstream-2024-04-02_0.mp3'>